In [1]:
import numpy as np
import pandas as pd

In [2]:
from io import StringIO

In [3]:
import re

# Table 2: Comparison of batch generation strategies

In [4]:
!ls ../scenario_*/results/scenario_*__subseq_smpl_strategy.csv

../scenario_age_pred/results/scenario_age_pred__subseq_smpl_strategy.csv
../scenario_bowl2019/results/scenario_bowl2019__subseq_smpl_strategy.csv
../scenario_gender/results/scenario_gender__subseq_smpl_strategy.csv
../scenario_x5/results/scenario_x5__subseq_smpl_strategy.csv


In [5]:
!ls ../scenario_rosbank/results/scenario_rosbank__subseq_smpl_strategy.txt

../scenario_rosbank/results/scenario_rosbank__subseq_smpl_strategy.txt


In [6]:
def read_csv(file_name, dataset_name):
    df = pd.read_fwf(file_name).iloc[2:, 1:3]
    metric_name = df.columns[1]
    df.columns = ['report_name', f'{dataset_name} ({metric_name})']
    df['report_name'] = df['report_name'].str.replace('.pickle', '', regex=True)
    df['report_name'] = df['report_name'].str.replace('emb__', '', regex=True)
    df = df.set_index('report_name')
    return df

def read_txt(file_name, dataset_name, kind='valid'):
    lines = open(file_name, 'r').readlines()
    ix_valid = np.where([True if re.findall(f'scores_valid', l) else False for l in lines])[0][0]
    ix_test = np.where([True if re.findall(f'scores_test', l) else False for l in lines])[0][0]
    ix_end = np.where([True if re.findall(f'---------------------------', l) else False for l in lines])[0][2]
    
    if kind == 'valid':
        ix_s = ix_valid
        ix_e = ix_test
    if kind == 'test':
        ix_s = ix_test
        ix_e = ix_end
    df = pd.read_fwf(StringIO('\n'.join(lines[ix_s + 2:ix_e]))).iloc[:, 1:3]
    metric_name = lines[ix_valid - 1].strip().split(' ')[1].strip('"')
    df.columns = ['report_name', f'{dataset_name} ({metric_name})']
    df['report_name'] = df['report_name'].str.replace('emb_mles__subseq_', '', regex=True)
    df['report_name'] = df['report_name'].str.replace('emb_mles__', '', regex=True)
    df = df.set_index('report_name')
    
    if kind == 'valid':
        return df
    
    df_m = df
#     if kind == 'test':
#     ix_s = ix_valid
#     ix_e = ix_test
    ix_s = ix_test
    ix_e = ix_end
    df = pd.read_fwf(StringIO('\n'.join(lines[ix_s + 2:ix_e]))).iloc[:, [1, 6]]
    df.columns = ['report_name', 'std']
    df['report_name'] = df['report_name'].str.replace('emb_mles__subseq_', '', regex=True)
    df['report_name'] = df['report_name'].str.replace('emb_mles__', '', regex=True)
    df = df.set_index('report_name')
    df_m[f'{dataset_name} ({metric_name})'] = df_m[f'{dataset_name} ({metric_name})'].astype(str) + \
    ' ± ' + df['std'].astype(str)

    return df_m

In [7]:
pd.concat([
    read_csv('../scenario_age_pred/results/scenario_age_pred__subseq_smpl_strategy.csv', 'Age'),
    read_txt('../scenario_rosbank/results/scenario_rosbank__subseq_smpl_strategy.txt', 'Churn'),
    read_csv('../scenario_bowl2019/results/scenario_bowl2019__subseq_smpl_strategy.csv', 'Assess'),
    read_csv('../scenario_x5/results/scenario_x5__subseq_smpl_strategy.csv', 'Retail'),
    read_csv('../scenario_gender/results/scenario_gender__subseq_smpl_strategy.csv', 'Gender'),
], axis=1)

,Age (oof_accuracy),Churn (auroc),Assess (oof_accuracy),Retail (oof_accuracy),Gender (oof_rocauc_score)
report_name,,,,,
SampleRandom,0.6130,0.820,0.5630,0.5225,0.8601
SplitRandom,0.6193,0.819,NaN,0.5047,0.8476
mles_embeddings,0.6388,0.823,0.6184,0.5420,0.8764
SampleRandom_short,NaN,NaN,0.5630,NaN,NaN


# Table 3: Comparison of encoder types

In [8]:
!ls ../scenario_*/results/scenario_*__encoder_types.csv

../scenario_age_pred/results/scenario_age_pred__encoder_types.csv
../scenario_bowl2019/results/scenario_bowl2019__encoder_types.csv
../scenario_gender/results/scenario_gender__encoder_types.csv
../scenario_x5/results/scenario_x5__encoder_types.csv


In [9]:
!ls ../scenario_rosbank/results/scenario_rosbank__encoder_types.txt

../scenario_rosbank/results/scenario_rosbank__encoder_types.txt


In [10]:
pd.concat([
    read_csv('../scenario_age_pred/results/scenario_age_pred__encoder_types.csv', 'Age'),
    read_txt('../scenario_rosbank/results/scenario_rosbank__encoder_types.txt', 'Churn'),
    read_csv('../scenario_bowl2019/results/scenario_bowl2019__encoder_types.csv', 'Assess'),
    read_csv('../scenario_x5/results/scenario_x5__encoder_types.csv', 'Retail'),
    read_csv('../scenario_gender/results/scenario_gender__encoder_types.csv', 'Gender'),
], axis=1)

,Age (oof_accuracy),Churn (auroc),Assess (oof_accuracy),Retail (oof_accuracy),Gender (oof_rocauc_score)
report_name,,,,,
encoder_lstm,0.6211,0.823,0.6198,0.5356,0.8573
encoder_transf,0.6215,0.780,NaN,0.4986,0.8601
mles_embeddings,0.6388,0.823,0.6184,0.5420,0.8764
encoder_gru,NaN,0.812,0.6184,NaN,NaN
encoder_transf_bs064_4head_64hs_4layers,NaN,NaN,0.5425,NaN,NaN
encoder_transf_bs064_4head_64hs_8layers,NaN,NaN,0.5357,NaN,NaN


# Table 4: Comparison of contrastive learning losses

In [11]:
!ls ../scenario_*/results/scenario_*__loss.csv

../scenario_age_pred/results/scenario_age_pred__loss.csv
../scenario_bowl2019/results/scenario_bowl2019__loss.csv
../scenario_gender/results/scenario_gender__loss.csv
../scenario_x5/results/scenario_x5__loss.csv


In [12]:
!ls ../scenario_rosbank/results/scenario_rosbank__loss.txt

../scenario_rosbank/results/scenario_rosbank__loss.txt


In [13]:
pd.concat([
    read_csv('../scenario_age_pred/results/scenario_age_pred__loss.csv', 'Age'),
    read_txt('../scenario_rosbank/results/scenario_rosbank__loss.txt', 'Churn'),
    read_csv('../scenario_bowl2019/results/scenario_bowl2019__loss.csv', 'Assess'),
    read_csv('../scenario_x5/results/scenario_x5__loss.csv', 'Retail'),
    read_csv('../scenario_gender/results/scenario_gender__loss.csv', 'Gender'),
], axis=1).sort_index()

,Age (oof_accuracy),Churn (auroc),Assess (oof_accuracy),Retail (oof_accuracy),Gender (oof_rocauc_score)
report_name,,,,,
loss_binomialdeviance,0.6214,0.769,NaN,0.5349,0.8510
loss_binomialdeviance_C_1.0_alpha_1.0_beta_0.4,NaN,NaN,0.5888,NaN,NaN
loss_binomialdeviance_C_6.0_alpha_0.4_beta_0.7,NaN,NaN,0.5857,NaN,NaN
loss_contrastive_margin_0.3,NaN,0.820,NaN,NaN,NaN
loss_contrastive_margin_0.5,0.6388,0.823,0.6184,0.5420,0.8764
loss_contrastive_margin_0.8,NaN,0.822,NaN,NaN,NaN
loss_contrastive_margin_1.0,0.6357,NaN,0.5973,0.5414,0.8672
loss_histogramloss,0.6321,0.815,0.6148,0.5335,0.8592
loss_margin_0.2_beta_0.4,0.6378,0.823,0.6119,0.5409,0.8746


# Table 5: Comparison of negative sampling strategies

In [14]:
!ls ../scenario_*/results/scenario_*__smpl_strategy.csv

../scenario_age_pred/results/scenario_age_pred__smpl_strategy.csv
../scenario_bowl2019/results/scenario_bowl2019__smpl_strategy.csv
../scenario_gender/results/scenario_gender__smpl_strategy.csv
../scenario_x5/results/scenario_x5__smpl_strategy.csv


In [15]:
!ls ../scenario_rosbank/results/scenario_rosbank__smpl_strategy.txt

../scenario_rosbank/results/scenario_rosbank__smpl_strategy.txt


In [16]:
pd.concat([
    read_csv('../scenario_age_pred/results/scenario_age_pred__smpl_strategy.csv', 'Age'),
    read_txt('../scenario_rosbank/results/scenario_rosbank__smpl_strategy.txt', 'Churn'),
    read_csv('../scenario_bowl2019/results/scenario_bowl2019__smpl_strategy.csv', 'Assess'),
    read_csv('../scenario_x5/results/scenario_x5__smpl_strategy.csv', 'Retail'),
    read_csv('../scenario_gender/results/scenario_gender__smpl_strategy.csv', 'Gender'),
], axis=1)

,Age (oof_accuracy),Churn (auroc),Assess (oof_accuracy),Retail (oof_accuracy),Gender (oof_rocauc_score)
report_name,,,,,
smpl_strategy_AllPositivePair,0.6262,0.815,0.5933,0.5301,0.8421
smpl_strategy_DistanceWeightedPair,0.6289,0.821,0.6031,0.5355,0.8474
smpl_strategy_HardNegativePair_neg_count_2,0.6377,0.821,0.6184,0.5418,0.8732
smpl_strategy_HardNegativePair_neg_count_5,0.6388,0.823,0.6150,0.5366,0.8764
smpl_strategy_HardNegativePair_neg_count_9,0.6364,0.823,0.6092,0.5420,0.8717
mles_embeddings,0.6388,0.823,0.6184,0.5420,0.8764


# Table 6: Quality of unsupervised embeddings as features for the downstream task

In [17]:
!ls ../scenario_*/results/scenario_*_baselines_unsupervised.txt

../scenario_age_pred/results/scenario_age_pred_baselines_unsupervised.txt
../scenario_bowl2019/results/scenario_bowl2019_baselines_unsupervised.txt
../scenario_gender/results/scenario_gender_baselines_unsupervised.txt
../scenario_rosbank/results/scenario_rosbank_baselines_unsupervised.txt
../scenario_x5/results/scenario_x5_baselines_unsupervised.txt


In [18]:
pd.concat([
    read_txt('../scenario_age_pred/results/scenario_age_pred_baselines_unsupervised.txt', 'Age', 'test'),
    read_txt('../scenario_rosbank/results/scenario_rosbank_baselines_unsupervised.txt', 'Churn', 'test'),
    read_txt('../scenario_bowl2019/results/scenario_bowl2019_baselines_unsupervised.txt', 'Assess', 'test'),
    read_txt('../scenario_x5/results/scenario_x5_baselines_unsupervised.txt', 'Retail', 'test'),
    read_txt('../scenario_gender/results/scenario_gender_baselines_unsupervised.txt', 'Gender', 'test'),
], axis=1)

,Age (accuracy),Churn (auroc),Assess (accuracy),Retail (accuracy),Gender (auroc)
report_name,,,,,
baseline,0.631 ± 0.003,0.825 ± 0.004,0.602 ± 0.005,0.547 ± 0.001,0.875 ± 0.003
cpc_embeddings,0.594 ± 0.002,0.802 ± 0.003,0.588 ± 0.002,0.525 ± 0.001,0.845 ± 0.002
mles_embeddings,0.638 ± 0.007,0.843 ± 0.003,0.601 ± 0.002,0.539 ± 0.001,0.89 ± 0.003
nsp_embeddings,0.622 ± 0.004,0.83 ± 0.004,0.581 ± 0.003,0.425 ± 0.002,0.837 ± 0.004
rtd_embeddings,0.632 ± 0.002,0.801 ± 0.004,0.58 ± 0.003,0.52 ± 0.001,0.86 ± 0.003
sop_embeddings,0.493 ± 0.002,0.782 ± 0.005,0.577 ± 0.002,0.428 ± 0.001,0.749 ± 0.009


# Table 7: Quality of the pre-trained model on the downstream tasks

In [19]:
!ls ../scenario_*/results/scenario_*_baselines_supervised.txt

../scenario_age_pred/results/scenario_age_pred_baselines_supervised.txt
../scenario_bowl2019/results/scenario_bowl2019_baselines_supervised.txt
../scenario_gender/results/scenario_gender_baselines_supervised.txt
../scenario_rosbank/results/scenario_rosbank_baselines_supervised.txt
../scenario_x5/results/scenario_x5_baselines_supervised.txt


In [20]:
pd.concat([
    read_txt('../scenario_age_pred/results/scenario_age_pred_baselines_supervised.txt', 'Age', 'test'),
    read_txt('../scenario_rosbank/results/scenario_rosbank_baselines_supervised.txt', 'Churn', 'test'),
    read_txt('../scenario_bowl2019/results/scenario_bowl2019_baselines_supervised.txt', 'Assess', 'test'),
    read_txt('../scenario_x5/results/scenario_x5_baselines_supervised.txt', 'Retail', 'test'),
    read_txt('../scenario_gender/results/scenario_gender_baselines_supervised.txt', 'Gender', 'test'),
], axis=1)

,Age (accuracy),Churn (auroc),Assess (accuracy),Retail (accuracy),Gender (auroc)
report_name,,,,,
cpc_finetuning,0.615 ± 0.009,0.81 ± 0.006,0.606 ± 0.004,0.549 ± 0.001,0.872 ± 0.002
mles_finetuning,0.644 ± 0.004,0.827 ± 0.004,0.615 ± 0.003,0.552 ± 0.001,0.89 ± 0.003
rtd_finetuning,0.635 ± 0.006,0.819 ± 0.005,0.586 ± 0.003,0.544 ± 0.002,0.881 ± 0.007
target_scores,0.628 ± 0.004,0.817 ± 0.009,0.602 ± 0.005,0.542 ± 0.001,0.856 ± 0.006
nsp_finetuning,NaN,0.818 ± 0.006,NaN,NaN,NaN
